# Chapter 10

In [1]:
%use s2

import java.util.Arrays

println("Chapter 10 demos")

Chapter 10 demos


In [2]:
println("Equality constraints")

// non-linear constraints
val a = GeneralEqualityConstraints(
                // the first equality constraint
        object : AbstractRealScalarFunction(3) { // the domain dimension
                    
            override fun evaluate(x: Vector): Double {
                val x1: Double = x.get(1)
                val x3: Double = x.get(3)

                val a1: Double = -x1 + x3 + 1
                return a1
            }
        },
                // the second equality constraint
        object : AbstractRealScalarFunction(3) { // the domain dimension
                    
            override fun evaluate(x: Vector): Double {
                val x1: Double = x.get(1)
                val x2: Double = x.get(2)

                val a2: Double = x1 * x1 + x2 * x2 - 2.0 * x1
                return a2
            }
        }
)

println(a)

Equality constraints
dev.nm.solver.multivariate.constrained.constraint.general.GeneralEqualityConstraints@367ea352


In [3]:
/** Example 10.2, p. 270. Practical Optimization: Algorithms and
    * Engineering Applications. Andreas Antoniou, Wu-Sheng Lu */
// linear constraints

val A: Matrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(1.0, -2.0, 3.0, 2.0),
        doubleArrayOf(0.0, 2.0, -1.0, 0.0),
        doubleArrayOf(2.0, -10.0, 9.0, 4.0)
    )
)
        
val b: Vector = DenseVector(4.0, 1.0, 5.0)
val A_eq = LinearEqualityConstraints(A, b)
println("original equality constraints: ")
println(A_eq)

// do SVD decomposition to reduce the eqaulity constraints
val svd: SVD = SVD(A, true)
val U: Matrix = svd.U()
println("\nU = ")
println(U)
val D: Matrix = svd.D()
println("\nD = ")
println(D)
val V: Matrix = svd.V()
println("\nV = ")
println(V)

// check if the original equality constraints are reducible
val epsilon: Double = 1e-8 // the precision parameter under which is considered 0
val isReducible: Boolean = A_eq.isReducible()
println(isReducible)
val r: Int = MatrixMeasure.rank(
        A,
        epsilon
)
println(String.format("rank of A = %d%n", r))

// construct a set of reduced constraints
// val A_eq_hat = A_eq.getReducedLinearEqualityConstraints() // TODO: Deprecated in java
// println("reduced equality constraints: ")
// println(A_eq_hat)

original equality constraints: 
3x4
	[,1] [,2] [,3] [,4] 
[1,] 1.000000, -2.000000, 3.000000, 2.000000, 
[2,] 0.000000, 2.000000, -1.000000, 0.000000, 
[3,] 2.000000, -10.000000, 9.000000, 4.000000, =
[4.000000, 1.000000, 5.000000] 

U = 
3x3
	[,1] [,2] [,3] 
[1,] 0.271659, 0.800304, -0.534522, 
[2,] -0.136451, 0.581828, 0.801784, 
[3,] 0.952671, -0.144875, 0.267261, 

D = 
3x3
	[,1] [,2] [,3] 
[1,] 14.879768, 0.000000, 0.000000, 
[2,] 0.000000, 1.610126, 0.000000, 
[3,] 0.000000, 0.000000, 0.000000, 

V = 
4x3
	[,1] [,2] [,3] 
[1,] 0.146306, 0.317089, -0.936741, 
[2,] -0.695100, 0.628398, 0.112540, 
[3,] 0.640162, 0.319980, 0.225081, 
[4,] 0.292612, 0.634179, 0.243290, 
true
rank of A = 2



In [4]:
println("Inequality constraints")

val c_gr = GeneralGreaterThanConstraints(
            object : AbstractBivariateRealFunction() {

            override fun evaluate(x1: Double, x2: Double): Double {
                val c: Double = x2 - x1 * x1
                return c
            }
        })

println(c_gr)

Inequality constraints
dev.nm.solver.multivariate.constrained.constraint.general.GeneralGreaterThanConstraints@152aa6a1


In [5]:
val c_less = GeneralLessThanConstraints(
    object : AbstractBivariateRealFunction() {
            
    override fun evaluate(x1: Double, x2: Double): Double {
        val c: Double = x1 * x1 - x2
        return c
    }
})

println(c_less)

dev.nm.solver.multivariate.constrained.constraint.general.GeneralLessThanConstraints@6e480b3


In [6]:
// w_i >= 0 mean no short selling
val A: Matrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(1.0, 0.0, 0.0),
        doubleArrayOf(0.0, 1.0, 0.0),
        doubleArrayOf(0.0, 0.0, 1.0),
    )
)
val b1: Vector = DenseVector(0.0, 0.0, 0.0) // 3 stocks
val no_short_selling1 = LinearGreaterThanConstraints(A, b1)// w >= 0
println(no_short_selling1)

3x3
	[,1] [,2] [,3] 
[1,] 1.000000, 0.000000, 0.000000, 
[2,] 0.000000, 1.000000, 0.000000, 
[3,] 0.000000, 0.000000, 1.000000, >=
[0.000000, 0.000000, 0.000000] 


In [7]:
val no_short_selling2 = LowerBoundConstraints(3, 0.0);
println(no_short_selling2);

Line_10.jupyter-kts (1:47 - 48) The integer literal does not conform to the expected type RealScalarFunction!

In [8]:
val no_short_selling3 = NonNegativityConstraints(3);
println(no_short_selling3);

Line_11.jupyter-kts (1:50 - 51) The integer literal does not conform to the expected type RealScalarFunction!

In [9]:
// w_i <= 0.2
val A: Matrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(1.0, 0.0, 0.0),
        doubleArrayOf(0.0, 1.0, 0.0),
        doubleArrayOf(0.0, 0.0, 1.0),
    )
)
val b2: Vector = DenseVector(0.2, 0.2, 0.2) // 3 stocks
val maximum_exposure = LinearLessThanConstraints(A, b2)// w >= 0
println(maximum_exposure)

3x3
	[,1] [,2] [,3] 
[1,] 1.000000, 0.000000, 0.000000, 
[2,] 0.000000, 1.000000, 0.000000, 
[3,] 0.000000, 0.000000, 1.000000, <=
[0.200000, 0.200000, 0.200000] 


In [10]:
println("LP problems of different forms")

// construct an LP problem in standard form
val problem1 = LPStandardProblem(
        DenseVector(-1.0, -1.0, 0.0, 0.0), // c
        LinearEqualityConstraints(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(7.0, 1.0, 1.0, 0.0),
                        doubleArrayOf(-1.0, 1.0, 0.0, 1.0)
                    )
                ),
                DenseVector(15.0, 1.0) // b
        ))
println(problem1)

LP problems of different forms
min. objective:
[-1.000000, -1.000000, 0.000000, 0.000000] 
less-than-or-equal-to inequalities:
4x4
	[,1] [,2] [,3] [,4] 
[1,] -7.000000, -1.000000, -1.000000, -0.000000, 
[2,] 1.000000, -1.000000, -0.000000, -1.000000, 
[3,] 7.000000, 1.000000, 1.000000, 0.000000, 
[4,] -1.000000, 1.000000, 0.000000, 1.000000, <=
[-15.000000, -1.000000, 15.000000, 1.000000] 
equalities:
free variables:



In [11]:
// construct an LP problem in canonical form 1
val problem2 = LPCanonicalProblem1(
                DenseVector(-1.0, -1.0, 0.0, 0.0), // c
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(7.0, 1.0, 1.0, 0.0),
                        doubleArrayOf(-1.0, 1.0, 0.0, 1.0),
                        doubleArrayOf(-7.0, -1.0, -1.0, 0.0),
                        doubleArrayOf(1.0, -1.0, 0.0, -1.0),
                    )
                ),
                DenseVector(15.0, 1.0, -15.0, -1.0) // b
        )
println(problem2)

min. objective:
[-1.000000, -1.000000, 0.000000, 0.000000] 
less-than-or-equal-to inequalities:
4x4
	[,1] [,2] [,3] [,4] 
[1,] -7.000000, -1.000000, -1.000000, -0.000000, 
[2,] 1.000000, -1.000000, -0.000000, -1.000000, 
[3,] 7.000000, 1.000000, 1.000000, -0.000000, 
[4,] -1.000000, 1.000000, -0.000000, 1.000000, <=
[-15.000000, -1.000000, 15.000000, 1.000000] 
equalities:
free variables:



In [12]:
// construct an LP problem in canonical form 2
val problem3 = LPCanonicalProblem2(
                DenseVector(-1.0, -1.0, 0.0, 0.0), // c
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(-7.0, -1.0, -1.0, 0.0),
                        doubleArrayOf(1.0, -1.0, 0.0, -1.0),
                        doubleArrayOf(7.0, 1.0, 1.0, 0.0),
                        doubleArrayOf(-1.0, 1.0, 0.0, 1.0),
                    )
                ),
                DenseVector(-15.0, -1.0, 15.0, 1.0) // b
        )
println(problem3)

b >= 0
java.lang.IllegalArgumentException: b >= 0
	at dev.nm.misc.ArgumentAssertion.assertTrue(nub:249)
	at dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.lp.problem.LPCanonicalProblem2.break(feb:99)
	at dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.lp.problem.LPCanonicalProblem2.<init>(feb:51)
	at Line_15.<init>(Line_15.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluato

In [13]:
// construct an LP problem in canonical form 2
val problem4 = LPCanonicalProblem2(
                DenseVector(1.0, 1.0), // c
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(1.0, 1.0),
                        doubleArrayOf(1.0, 2.0),
                        doubleArrayOf(0.0, 3.0)
                    )
                ),
                DenseVector(150.0, 170.0, 180.0) // b
        )
println(problem4)

min. objective:
[1.000000, 1.000000] 
less-than-or-equal-to inequalities:
3x2
	[,1] [,2] 
[1,] 1.000000, 1.000000, 
[2,] 1.000000, 2.000000, 
[3,] 0.000000, 3.000000, <=
[150.000000, 170.000000, 180.000000] 
equalities:
free variables:



In [14]:
/**
* Example 3-1-1.
*
* Linear Programming with MATLAB
* by Michael C. Ferris, Olvi L. Mangasarian, Stephen J. Wright.
*/

println("Example 3.1.1")

// construct an LP problem
val problem = LPCanonicalProblem1(
        DenseVector(3.0, -6.0), // c
        DenseMatrix(
            arrayOf(
                doubleArrayOf(1.0, 2.0),
                doubleArrayOf(2.0, 1.0),
                doubleArrayOf(1.0, -1.0),
                doubleArrayOf(1.0, -4.0),
                doubleArrayOf(-4.0, 1.0)
            )
        ),
        DenseVector(-1.0, 0.0, -1.0, -13.0, -23.0) // b
)

var tableau = SimplexTable(problem)
println(tableau)

example 3.1.1
6x3
                                NON_BASIC,1                     NON_BASIC,2                     B,2147483647                    
BASIC,1                         1.000000                        2.000000                        1.000000                        
BASIC,2                         2.000000                        1.000000                        -0.000000                       
BASIC,3                         1.000000                        -1.000000                       1.000000                        
BASIC,4                         1.000000                        -4.000000                       13.000000                       
BASIC,5                         -4.000000                       1.000000                        23.000000                       
COST,2147483647                 3.000000                        -6.000000                       0.000000                        



In [15]:
tableau = tableau.swap(3, 2)
println(tableau)

6x3
                                NON_BASIC,1                     BASIC,3                         B,2147483647                    
BASIC,1                         3.000000                        -2.000000                       3.000000                        
BASIC,2                         3.000000                        -1.000000                       1.000000                        
NON_BASIC,2                     1.000000                        -1.000000                       1.000000                        
BASIC,4                         -3.000000                       4.000000                        9.000000                        
BASIC,5                         -3.000000                       -1.000000                       24.000000                       
COST,2147483647                 -3.000000                       6.000000                        -6.000000                       



In [16]:
tableau = tableau.swap(4, 1)
println(tableau)

6x3
                                BASIC,4                         BASIC,3                         B,2147483647                    
BASIC,1                         -1.000000                       2.000000                        12.000000                       
BASIC,2                         -1.000000                       3.000000                        10.000000                       
NON_BASIC,2                     -0.333333                       0.333333                        4.000000                        
NON_BASIC,1                     -0.333333                       1.333333                        3.000000                        
BASIC,5                         1.000000                        -5.000000                       15.000000                       
COST,2147483647                 1.000000                        2.000000                        -15.000000                      



In [17]:
/**
* Example 3-4-1.
*
* Linear Programming with MATLAB
* by Michael C. Ferris, Olvi L. Mangasarian, Stephen J. Wright.
*/

println("Phase 1 procedure")

// construct an LP problem
problem = LPProblemImpl1(
        DenseVector(4.0, 5.0), // c
        LinearGreaterThanConstraints(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(1.0, 1.0),
                        doubleArrayOf(1.0, 2.0),
                        doubleArrayOf(4.0, 2.0),
                        doubleArrayOf(-1.0, -1.0),
                        doubleArrayOf(-1.0, 1.0)
                    )
                ),
                DenseVector(-1.0, 1.0, 8.0, -3.0, 1.0)), // b
        null, // less-than constraints
        null, // equality constraints
        null) // box constraints
table0 = SimplexTable(problem)
println("tableau for the original problem:")
println(table0)

val phase1 = FerrisMangasarianWrightPhase1(table0)
val table1 = phase1.process()
println("tableau for the phase 1 problem:")
println(table1)

println(String.format("minimum = %f%n", table1.minimum()))
println(String.format("minimizer = %s%n", table1.minimizer()))

Line_20.jupyter-kts (11:1 - 8) Val cannot be reassigned
Line_20.jupyter-kts (11:11 - 26:14) Type mismatch: inferred type is LPProblemImpl1 but LPCanonicalProblem1 was expected
Line_20.jupyter-kts (27:1 - 7) Unresolved reference: table0
Line_20.jupyter-kts (29:9 - 15) Unresolved reference: table0
Line_20.jupyter-kts (31:44 - 50) Unresolved reference: table0

In [18]:
/**
* Example 3-4-1.
*
* Linear Programming with MATLAB
* by Michael C. Ferris, Olvi L. Mangasarian, Stephen J. Wright.
*/

println("Solving an LP problem")

// construct an LP problem
var problem = LPProblemImpl1(
        DenseVector(4.0, 5.0), // c
        LinearGreaterThanConstraints(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(1.0, 1.0),
                        doubleArrayOf(1.0, 2.0),
                        doubleArrayOf(4.0, 2.0),
                        doubleArrayOf(-1.0, -1.0),
                        doubleArrayOf(-1.0, 1.0)
                    )
                ),
                DenseVector(-1.0, 1.0, 8.0, -3.0, 1.0)), // b
        null, // less-than constraints
        null, // equality constraints
        null) // box constraints

// construct the simplex tableau for the LP problem
var table0 = SimplexTable(problem)
println("Simplex tableau for the problem:")
println(table0)

// solve the LP problem using the 2-phase algorithm
val solver = LPTwoPhaseSolver()
val solution = solver.solve(problem).minimizer() as LPBoundedMinimizer

println(String.format("minimum = %f%n", solution.minimum()))
println(String.format("minimizer = %s%n", solution.minimizer()))

Solving an LP problem
simplex tableau for the problem:
6x3
                                NON_BASIC,1                     NON_BASIC,2                     B,2147483647                    
BASIC,1                         1.000000                        1.000000                        1.000000                        
BASIC,2                         1.000000                        2.000000                        -1.000000                       
BASIC,3                         4.000000                        2.000000                        -8.000000                       
BASIC,4                         -1.000000                       -1.000000                       3.000000                        
BASIC,5                         -1.000000                       1.000000                        -1.000000                       
COST,2147483647                 4.000000                        5.000000                        0.000000                        

minimum = 14.000000

minimizer = [1.0

In [19]:
/**
* Example 11.1.
*
* Applied Integer Programming: Modeling and Solution
* by Der-San Chen, Robert G. Batson, Yu Dang.
*/

println("Solving an LP problem")

// construct an LP problem
val problem = LPProblemImpl1(
        DenseVector(-5.0, 2.0), // c
        LinearGreaterThanConstraints(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(1.0, 3.0)
                    )
                ),
                DenseVector(9.0)), // b1
        LinearLessThanConstraints(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(-1.0, 2.0),
                        doubleArrayOf(3.0, 2.0)
                    )
                ),
                DenseVector(5.0, 19.0)), // b2
        null,
        null)

// construct the simplex tableau for the LP problem
val table0 = SimplexTable(problem)
println("Simplex tableau for the problem:")
println(table0)

// solve the LP problem using the 2-phase algorithm
val solver = LPTwoPhaseSolver()
val solution: LPMinimizer = solver.solve(problem).minimizer()

println(String.format("minimum = %f%n", solution.minimum()))
println(String.format("minimizer = %s%n", solution.minimizer()))

Solving an LP problem
simplex tableau for the problem:
4x3
                                NON_BASIC,1                     NON_BASIC,2                     B,2147483647                    
BASIC,1                         1.000000                        3.000000                        -9.000000                       
BASIC,2                         1.000000                        -2.000000                       5.000000                        
BASIC,3                         -3.000000                       -2.000000                       19.000000                       
COST,2147483647                 -5.000000                       2.000000                        0.000000                        

minimum = -25.571429

minimizer = [5.571429, 1.142857] 



In [20]:
/**
* Example 3-6-13 (b), pp. 84.
*
* Linear Programming with MATLAB
* by Michael C. Ferris, Olvi L. Mangasarian, Stephen J. Wright.
*
* This case is found infeasible during phase 1.
*/

println("Solving an LP problem")

// construct an LP problem
fun problem(): LPProblem {
    // min c'x
    val c: Vector = DenseVector(2.0, -1.0)

    // the constraints
    // Ax >= b
    val greaterThanConstraints = LinearGreaterThanConstraints(
        DenseMatrix(arrayOf(doubleArrayOf(1.0, 0.0))), // A
        DenseVector(-6.0)) // b
    val lessThanConstraints: LinearLessThanConstraints? = null // no less than constraints
    // Ax = b
    val equalityConstraints = LinearEqualityConstraints(
        DenseMatrix(arrayOf(doubleArrayOf(-1.0, 0.0))), // A
        DenseVector(-4.0)) // b
    // the whole plane
    val boxConstraints = BoxConstraints(
        2,
        BoxConstraints.Bound(2, Double.NEGATIVE_INFINITY, Double.POSITIVE_INFINITY)
    )

    // construct an LP problem with constraints
    val problem: LPProblem = LPProblemImpl1(
        c,
        greaterThanConstraints,
        lessThanConstraints,
        equalityConstraints,
        boxConstraints
    ) // x2 is free
    return problem
}

// construct the simplex tableau for the LP problem
val table0 = SimplexTable(problem())
println("Simplex tableau for the problem:")
println(table0)

// solve the LP problem using the 2-phase algorithm
val solver = LPTwoPhaseSolver()
val solution: LPMinimizer = solver.solve(problem()).minimizer()

println(String.format("minimum = %f%n", solution.minimum()))
println(String.format("minimizer = %s%n", solution.minimizer()))

Solving an LP problem
simplex tableau for the problem:
3x3
                                NON_BASIC,1                     FREE,2                          B,2147483647                    
BASIC,1                         1.000000                        0.000000                        6.000000                        
EQUALITY,1                      -1.000000                       0.000000                        4.000000                        
COST,2147483647                 2.000000                        -1.000000                       0.000000                        

minimum = -Infinity

minimizer = [0.000000, 1.000000] 



In [21]:
/**
* Example 3-6-13 (b), pp. 84.
*
* Linear Programming with MATLAB
* by Michael C. Ferris, Olvi L. Mangasarian, Stephen J. Wright.
*
* This case is found infeasible during phase 1.
*/

println("Solving an LP problem")

// construct an LP problem
fun problem(): LPProblem {
    // min c'x
    val c: Vector = DenseVector(2.0, -1.0)

    // the constraints
    // Ax >= b
    val greaterThanConstraints = LinearGreaterThanConstraints(
        DenseMatrix(arrayOf(doubleArrayOf(1.0, 0.0))), // A
        DenseVector(6.0)) // b
    val lessThanConstraints: LinearLessThanConstraints? = null // no less than constraints
    // Ax = b
    val equalityConstraints = LinearEqualityConstraints(
        DenseMatrix(arrayOf(doubleArrayOf(-1.0, 0.0))), // A
        DenseVector(-4.0)) // b
    // the whole plane
    val boxConstraints = BoxConstraints(
        2,
        BoxConstraints.Bound(2, Double.NEGATIVE_INFINITY, Double.POSITIVE_INFINITY)
    )

    // construct an LP problem with constraints
    val problem: LPProblem = LPProblemImpl1(
        c,
        greaterThanConstraints,
        lessThanConstraints,
        equalityConstraints,
        boxConstraints
    ) // x2 is free
    return problem
}

// construct the simplex tableau for the LP problem
val table0 = SimplexTable(problem())
println("Simplex tableau for the problem:")
println(table0)

// solve the LP problem using the 2-phase algorithm
val solver = LPTwoPhaseSolver()
val solution: LPMinimizer = solver.solve(problem()).minimizer()

Solving an LP problem
simplex tableau for the problem:
3x3
                                NON_BASIC,1                     FREE,2                          B,2147483647                    
BASIC,1                         1.000000                        0.000000                        -6.000000                       
EQUALITY,1                      -1.000000                       0.000000                        4.000000                        
COST,2147483647                 2.000000                        -1.000000                       0.000000                        



null
dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.lp.exception.LPInfeasible
	at dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.lp.simplex.FerrisMangasarianWrightPhase1.process(gbb:148)
	at dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.lp.simplex.solver.LPTwoPhaseSolver$Solution.<init>(ilb:82)
	at dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.lp.simplex.solver.LPTwoPhaseSolver$Solution.<init>(ilb:57)
	at dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.lp.simplex.solver.LPTwoPhaseSolver.solve(ilb:100)
	at dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.lp.simplex.solver.LPTwoPhaseSolver.solve(ilb:26)
	at Line_24.<init>(Line_24.jupyter-kts:51)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newIn

In [22]:
/**
* Example 11.1.
*
* Applied Integer Programming: Modeling and Solution
* by Der-San Chen, Robert G. Batson, Yu Dang.
*/

println("Solving an LP problem")

// construct an LP problem
val problem = LPProblemImpl1(
        DenseVector(-5.0, 2.0), // c
        LinearGreaterThanConstraints(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(1.0, 3.0)
                    )
                ),
                DenseVector(9.0)), // b1
        LinearLessThanConstraints(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(-1.0, 2.0),
                        doubleArrayOf(3.0, 2.0)
                    )
                ),
                DenseVector(5.0, 19.0)), // b2
        null,
        null)

// solve the LP problem using the algebraic LP solver
val solver = LPRevisedSimplexSolver(1e-8)
val solution: LPMinimizer = solver.solve(problem).minimizer()

println(String.format("minimum = %f%n", solution.minimum()))
println(String.format("minimizer = %s%n", solution.minimizer()))

Solving an LP problem
minimum = -25.571429

minimizer = [5.571429, 1.142857] 



In [23]:
/**
* Example 3-6-13 (c), pp. 84.
*
* Linear Programming with MATLAB
* by Michael C. Ferris, Olvi L. Mangasarian, Stephen J. Wright.
*
* This case is founded unbound during phase 2.
*/

println("Solving an LP problem")

// construct an LP problem
fun problem(): LPProblem {
    // constraint: Ax >= b
    val greaterThanConstraints = LinearGreaterThanConstraints(
        DenseMatrix(arrayOf(doubleArrayOf(0.0, 1.0, -2.0, -1.0), doubleArrayOf(2.0, -1.0, -1.0, 4.0), doubleArrayOf(-1.0, 1.0, 0.0, -2.0))), // A
        DenseVector(-4.0, -5.0, -3.0) // b
    )

    // min c'x subject to the constraint
    val problem: LPProblem = LPProblemImpl1(
        DenseVector(1.0, -2.0, -4.0, 4.0), // c
        greaterThanConstraints,
        null
    )
    return problem
}

// construct the simplex tableau for the LP problem
val table0 = SimplexTable(problem())
println("Simplex tableau for the problem:")
println(table0)

// solve the LP problem using the 2-phase algorithm
val solver: LPTwoPhaseSolver = LPTwoPhaseSolver()
val solution: LPUnboundedMinimizer = solver.solve(problem()).minimizer() as LPUnboundedMinimizer

println(String.format("minimum = %f%n", solution.minimum()))
println(String.format("minimizer = %s%n", solution.minimizer()))
println(String.format("v = %s%n", solution.v()))

Solving an LP problem
simplex tableau for the problem:
4x5
                                NON_BASIC,1                     NON_BASIC,2                     NON_BASIC,3                     NON_BASIC,4                     B,2147483647                    
BASIC,1                         0.000000                        1.000000                        -2.000000                       -1.000000                       4.000000                        
BASIC,2                         2.000000                        -1.000000                       -1.000000                       4.000000                        5.000000                        
BASIC,3                         -1.000000                       1.000000                        0.000000                        -2.000000                       3.000000                        
COST,2147483647                 1.000000                        -2.000000                       -4.000000                       4.000000                        0.000000 

In [24]:
/**
* example 13.1 in
* Andreas Antoniou, Wu-Sheng Lu, "Algorithm 13.1, Quadratic and Convex
* Programming," Practical Optimization: Algorithms and Engineering
* Applications.
*
* @throws Exception
*/

println("Solving an QP problem with only equality constraints")

//construct the QP problem with only equality constraints
val H: DenseMatrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(1.0, 0.0, 0.0),
        doubleArrayOf(0.0, 1.0, 0.0),
        doubleArrayOf(0.0, 0.0, 0.0)
    )
)

val p: DenseVector = DenseVector(2.0, 1.0, -1.0)
val f = QuadraticFunction(H, p)
println("minimizing:")
println(f)

// equality constraints
val A: DenseMatrix = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.0, 1.0, 1.0)
        ))
val b: DenseVector = DenseVector(1.0)
val Aeq = LinearEqualityConstraints(A, b)

// solve a QP problem with only equality constraints
val soln = QPSimpleMinimizer.solve(f, Aeq)
val x: Vector = soln.minimizer()
val fx: Double = f.evaluate(x)
println(String.format("f(%s) = %f%n", x, fx))
println(String.format("is unique = %b%n", soln.isUnique()))

Solving an QP problem with only equality constraints
minimizing:
1/2 * x'3x3
	[,1] [,2] [,3] 
[1,] 1.000000, 0.000000, 0.000000, 
[2,] 0.000000, 1.000000, 0.000000, 
[3,] 0.000000, 0.000000, 0.000000, x + x'[2.000000, 1.000000, -1.000000] 
f([-2.000000, -2.000000, 3.000000] ) = -5.000000

is unique = true



In [25]:
/**
* example 16.4 in Jorge Nocedal, Stephen Wright
*
* There is a detailed trace (for debugging) on p. 475.
*/

println("Solving an QP problem")

// construct a quadratic function
val H: Matrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(2.0, 0.0),
        doubleArrayOf(0.0, 2.0),
    )
)

val p: Vector = DenseVector(-2.0, -5.0)
val f = QuadraticFunction(H, p)

// construct the linear inequality constraints

val A: Matrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(1.0, -2.0),
        doubleArrayOf(-1.0, -2.0),
        doubleArrayOf(-1.0, 2.0),
        doubleArrayOf(1.0, 0.0),
        doubleArrayOf(0.0, 1.0),
    )
)

val b: Vector = DenseVector(-2.0, -6.0, -2.0, 0.0, 0.0)
val greater = LinearGreaterThanConstraints(A, b)// x >= 0
// construct the QP problem
val problem = QPProblem(f, null, greater)

// construct a primal active set solver
val epsion: Double = Math.sqrt(PrecisionUtils.autoEpsilon(problem.f().Hessian()))
val solver1 = QPPrimalActiveSetMinimizer(
                epsion, // precision
                Integer.MAX_VALUE // max number of iterations
        )
// solve the QP problem using the primal active set method
val solution1: QPPrimalActiveSetMinimizer.Solution = solver1.solve(problem)
solution1.search(DenseVector(2.0, 0.0))
// print out the solution
println("minimizer = " + solution1.minimizer().minimizer())
println("minimum = " + solution1.minimum())

Solving an QP problem
minimizer = [1.400000, 1.700000] 
minimum = -6.450000000000001


In [26]:
// construct a quadratic function
val H: Matrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(2.0, 0.0),
        doubleArrayOf(0.0, 2.0),
    )
)

val p: Vector = DenseVector(-2.0, -5.0)
val f = QuadraticFunction(H, p)

// construct the linear inequality constraints

val A: Matrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(1.0, -2.0),
        doubleArrayOf(-1.0, -2.0),
        doubleArrayOf(-1.0, 2.0),
        doubleArrayOf(1.0, 0.0),
        doubleArrayOf(0.0, 1.0),
    )
)

val b: Vector = DenseVector(-2.0, -6.0, -2.0, 0.0, 0.0)
val greater = LinearGreaterThanConstraints(A, b)// x >= 0
// construct the QP problem
val problem = QPProblem(f, null, greater)

// construct a dual active set solver
val epsion: Double = Math.sqrt(PrecisionUtils.autoEpsilon(problem.f().Hessian()))
// solve the QP problem using the dual active set method
val solver2 = QPDualActiveSetMinimizer(
                epsion, // precision
                Integer.MAX_VALUE) // max number of iterations
val solution2: QPDualActiveSetMinimizer.Solution = solver2.solve(problem)
solution2.search()
// print out the solution
println("minimizer = " + solution2.minimizer().minimizer())
println("minimum = " + solution2.minimum())

minimizer = [1.400000, 1.700000] 
minimum = -6.450000000000001


In [27]:
println("Construct the primal and dual SDP problems")

// the primal SDP matrices
val C: SymmetricMatrix = SymmetricMatrix(
    arrayOf(
        doubleArrayOf(1.0),
        doubleArrayOf(2.0, 9.0),
        doubleArrayOf(3.0, 0.0, 7.0)
    )
)

val A1: SymmetricMatrix = SymmetricMatrix(
    arrayOf(
        doubleArrayOf(1.0),
        doubleArrayOf(0.0, 3.0),
        doubleArrayOf(1.0, 7.0, 5.0)
    )
)

val A2: SymmetricMatrix = SymmetricMatrix(
    arrayOf(
        doubleArrayOf(0.0),
        doubleArrayOf(2.0, 6.0),
        doubleArrayOf(8.0, 0.0, 4.0)
    )
)

// construct the primal SDP problem
val primal = SDPPrimalProblem(
                C,
                arrayOf<SymmetricMatrix>(A1, A2))

println(primal)

Construct the primal and dual SDP problems
dev.nm.solver.multivariate.constrained.convex.sdp.problem.SDPPrimalProblem@39b0ae06


In [28]:
// the dual SDP vector and matrices
val b: Vector = DenseVector(11.0, 19.0)
// construct the primal SDP problem
val dual = SDPDualProblem(
                b,
                C,
                arrayOf<SymmetricMatrix>(A1, A2))

println(dual)

dev.nm.solver.multivariate.constrained.convex.sdp.problem.SDPDualProblem@39049ed


In [29]:
/**
* p.465 in
* Andreas Antoniou, Wu-Sheng Lu
*/

println("Solving an SDP problem")

// define an SDP problem with matrices and vectors
val A0: SymmetricMatrix = SymmetricMatrix(
    arrayOf(
        doubleArrayOf(2.0),
        doubleArrayOf(-0.5, 2.0),
        doubleArrayOf(-0.6, 0.4, 3.0)
    )
)

val A1: SymmetricMatrix = SymmetricMatrix(
    arrayOf(
        doubleArrayOf(0.0),
        doubleArrayOf(1.0, 0.0),
        doubleArrayOf(0.0, 0.0, 0.0)
    )
)
        
val A2: SymmetricMatrix = SymmetricMatrix(
    arrayOf(
        doubleArrayOf(0.0),
        doubleArrayOf(0.0, 0.0),
        doubleArrayOf(1.0, 0.0, 0.0)
    )
)
        
val A3: SymmetricMatrix = SymmetricMatrix(
    arrayOf(
        doubleArrayOf(0.0),
        doubleArrayOf(0.0, 0.0),
        doubleArrayOf(0.0, 1.0, 0.0)
    )
)
        
val A4: SymmetricMatrix = A3.ONE()
val C: SymmetricMatrix = A0.scaled(-1.0)
val b: Vector = DenseVector(0.0, 0.0, 0.0, 1.0)
// construct an SDP problem
val problem = SDPDualProblem(
        b,
        C,
        arrayOf<SymmetricMatrix>(A1, A2, A3, A4))

// the initial feasible point
val X0: DenseMatrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(1.0 / 3.0, 0.0, 0.0),
        doubleArrayOf(0.0, 1.0 / 3.0, 0.0),
        doubleArrayOf(0.0, 0.0, 1.0 / 3.0)
    )
)

val y0: Vector = DenseVector(0.2, 0.2, 0.2, -4.0)

val S0: DenseMatrix = DenseMatrix(
    arrayOf(
        doubleArrayOf(2.0, 0.3, 0.4),
        doubleArrayOf(0.3, 2.0, -0.6),
        doubleArrayOf(0.4, -0.6, 1.0)
    )
)

// the initial central path
val path0: CentralPath = CentralPath(X0, y0, S0)

// solving SDP problem
val solver = PrimalDualPathFollowingMinimizer(
                0.9, // γ
                0.001) // ε
val solution: IterativeSolution<CentralPath> = solver.solve(problem)
val path: CentralPath = solution.search(path0)

//the solution from the textbook is accurate up to epsilon
//changing epsilon will change the answers
// primal solution
println("X = ")
println(path.X)
// dual solution
println("Y = ")
println(path.y)
println("S = ")
println(path.S)

Solving an SDP problem
X = 
3x3
	[,1] [,2] [,3] 
[1,] 0.000552, -0.000000, -0.000000, 
[2,] -0.000000, 0.000552, 0.000000, 
[3,] -0.000000, 0.000000, 0.998896, 
Y = 
[0.392921, 0.599995, -0.399992, -3.000469] 
S = 
3x3
	[,1] [,2] [,3] 
[1,] 1.000469, 0.107079, 0.000005, 
[2,] 0.107079, 1.000469, -0.000008, 
[3,] 0.000005, -0.000008, 0.000469, 


In [30]:
/**
* example 14.5 in
* Andreas Antoniou, Wu-Sheng Lu
*/

println("Solving an SOCP problem")

fun problem() : SOCPGeneralProblem {
    // min f'x
    val f: Vector = DenseVector(1.0, 0.0, 0.0, 0.0, 0.0)

    // The A's in the conic constraints.
    val A1t: Matrix = DenseMatrix(arrayOf(doubleArrayOf(0.0, -1.0, 0.0, 1.0, 0.0), doubleArrayOf(0.0, 0.0, 1.0, 0.0, -1.0)))
    val A2t: Matrix = DenseMatrix(arrayOf(doubleArrayOf(0.0, 0.5, 0.0, 0.0, 0.0), doubleArrayOf(0.0, 0.0, 1.0, 0.0, 0.0)))
    val A3t: Matrix = DenseMatrix(arrayOf(doubleArrayOf(0.0, 0.0, 0.0, -0.7071, -0.7071), doubleArrayOf(0.0, 0.0, 0.0, -0.3536, 0.3536)))

    // The b's in the conic constraints.
    val b1: Vector = f
    val b2: Vector = f.ZERO()
    val b3: Vector = f.ZERO()

    // The c's in the conic constraints.
    val c1: Vector = DenseVector(2) // zero
    val c2: Vector = DenseVector(-0.5, 0.0)
    val c3: Vector = DenseVector(4.2426, -0.7071)

    // The d's in the conic constraints.
    val d = doubleArrayOf(0.0, 1.0, 1.0)
    val constraints: kotlin.collections.List<SOCPGeneralConstraint> = java.util.Arrays.asList<SOCPGeneralConstraint>(
        SOCPGeneralConstraint(A1t.t(), c1, b1, d[0]),
        SOCPGeneralConstraint(A2t.t(), c2, b2, d[1]),
        SOCPGeneralConstraint(A3t.t(), c3, b3, d[2])
    )

    // The SOCP problem to be solved.
    val problem: SOCPGeneralProblem = SOCPGeneralProblem(
        f, 
        constraints) // ||Ax + b|| <= c'x + d
    return problem
}

// Uses interior point method to solve the given problem from a given starting point.
val x0: Vector = DenseVector(1.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0)
val s0: Vector = DenseVector(3.7, 1.0, -3.5, 1.0, 0.25, 0.5, 1.0, -0.35355, -0.1767)
val y0: Vector = DenseVector(-3.7, -1.5, -0.5, -2.5, -4.0)
// an initial guess
val soln0 = PrimalDualSolution(x0, s0, y0)

// construct an SOCP solver
val solver = PrimalDualInteriorPointMinimizer(
    1e-8,
    20) // max number of iterations

// solve the SOCP problem
val p = problem()
val soln: IterativeSolution<PrimalDualSolution> = solver.solve(p)
soln.search(soln0)

// primal solution
println("X = ")
println(soln.minimizer().x)
// dual solution
println("Y = ")
println(soln.minimizer().y)
println("S = ")
println(soln.minimizer().s)
println("minimum = " + soln.minimum())

Solving an SOCP problem
X = 
[1.000000, -0.292853, 0.956158, 1.121289, -0.585705, -0.956158, 1.288310, -0.883192, 0.937931] 
Y = 
[-1.707790, -2.044676, -0.852738, -2.544822, -2.485651] 
S = 
[1.707790, 0.500146, -1.632912, 1.000000, 0.522338, 0.852738, 1.000000, 0.685553, -0.728023] 
minimum = -1.7077904435826983


In [31]:
/**
* example 15.1 in
* Andreas Antoniou, Wu-Sheng Lu
*/

println("Solving an SQP problem with only equality constraints")

// objective function
val f: RealScalarFunction = object : RealScalarFunction {
            
        override fun evaluate(x: Vector): Double {
            val x1: Double = x.get(1)
            val x2: Double = x.get(2)
            val x3: Double = x.get(3)

            var fx: Double = -x1.pow(4.0)
            fx -= 2.0 * x2.pow(4.0)
            fx -= x3.pow(4.0)
            fx -= (x1 * x2).pow(2.0)
            fx -= (x1 * x3).pow(2.0)

            return fx
        }

        override fun dimensionOfDomain(): Int {
            return 3
        }

        override fun dimensionOfRange(): Int {
            return 1
        }
}

// equality constraints
val equality_constraints: EqualityConstraints = GeneralEqualityConstraints(
    object : RealScalarFunction {
                    
        override fun evaluate(x: Vector): Double {
            val x1: Double = x.get(1)
            val x2: Double = x.get(2)
            val x3: Double = x.get(3)

            var fx: Double = x1.pow(4.0)
            fx += x2.pow(4.0)
            fx += x3.pow(4.0)
            fx -= 25.0

            return fx // a1
        }

        override fun dimensionOfDomain(): Int {
            return 3
        }

        override fun dimensionOfRange(): Int {
            return 1
        }
    },
    object : RealScalarFunction {
                    
        override fun evaluate(x: Vector): Double {
            val x1: Double = x.get(1)
            val x2: Double = x.get(2)
            val x3: Double = x.get(3)

            var fx: Double = 8.0 * x1.pow(2.0)
            fx += 14.0 * x2.pow(2.0)
            fx += 7.0 * x3.pow(2.0)
            fx -= 56.0

            return fx // a2
        }

        override fun dimensionOfDomain(): Int {
            return 3
        }

        override fun dimensionOfRange(): Int {
            return 1
        }
})

// construct an SQP solver
val solver = SQPActiveSetOnlyEqualityConstraint1Minimizer(
    object : SQPActiveSetOnlyEqualityConstraint1Minimizer.VariationFactory {
        override fun newVariation(
        f: RealScalarFunction?,
        equal: EqualityConstraints?
        ): SQPASEVariation {
            val impl = SQPASEVariation2(100.0, 0.01, 10)
            impl.set(f, equal)
            return impl
        }
    },
    1e-8, // epsilon, threshold
    20) // max number of iterations
// solving an SQP problem
val solution: IterativeSolution<Vector>
        = solver.solve(f, equality_constraints)
val x: Vector = solution.search(
        DenseVector(3.0, 1.5, 3.0), // x0
        DenseVector(-1.0, -1.0)) // λ0
val fx: Double = f.evaluate(x)
// print out the solution
println("x = " + x)
println("fx = " + fx)

Solving an SQP problem with only equality constraints
x = [1.874065, -0.465820, 1.884720] 
fx = -38.28482786994784


In [32]:
/**
* example 15.2 in
* Andreas Antoniou, Wu-Sheng Lu
*/

println("Solving an SQP problem")

// objective function
val f: RealScalarFunction = object : RealScalarFunction {
            
    override fun evaluate(x: Vector): Double {
        val x1: Double = x.get(1)
        val x2: Double = x.get(2)
        val x3: Double = x.get(3)
        val x4: Double = x.get(4)

        var fx: Double = (x1 - x3) * (x1 - x3)
        fx += (x2 - x4) * (x2 - x4)
        fx /= 2

        return fx
    }

    override fun dimensionOfDomain(): Int {
        return 4
    }

    override fun dimensionOfRange(): Int {
        return 1
    }
}

// inequality constraints
val greater = GeneralGreaterThanConstraints(
                        // c1
            object : RealScalarFunction {
                    
            override fun evaluate(x: Vector): Double {
                val x1: Double = x.get(1)
                val x2: Double = x.get(2)

                val x12: Matrix = DenseMatrix(
                    arrayOf(
                        doubleArrayOf(x1, x2),
                        doubleArrayOf(2.0, 1.0)
                    )
                )         

                val A: Matrix = DenseMatrix(
                    arrayOf(
                        doubleArrayOf(0.25, 0.0),
                        doubleArrayOf(0.0, 1.0)
                    )
                )
                                        
                val B: Matrix = DenseMatrix(
                    arrayOf(
                        doubleArrayOf(0.5, 0.0),
                        doubleArrayOf(2.0, 1.0)
                    )
                )

                var FX: Matrix = x12.t().multiply(A).multiply(x12)
                FX = FX.scaled(-1.0)
                FX = FX.add(x12.t().multiply(B))

                var fx: Double = FX.get(1, 1)
                fx += 0.75

                return fx
            }

            override fun dimensionOfDomain(): Int {
                return 4
            }

            override fun dimensionOfRange(): Int {
                return 1
            }
        },
                        // c2
            object : RealScalarFunction {

            override fun evaluate(x: Vector): Double {
                val x3: Double = x.get(3)
                val x4: Double = x.get(4)

                val x34: Matrix = DenseMatrix(
                    arrayOf(
                        doubleArrayOf(x3, x4),
                        doubleArrayOf(2.0, 1.0)
                    )
                )

                val A: Matrix = DenseMatrix(
                    arrayOf(
                        doubleArrayOf(5.0, 3.0),
                        doubleArrayOf(3.0, 5.0)
                    )
                )

                val B: Matrix = DenseMatrix(
                    arrayOf(
                        doubleArrayOf(11.0 / 2.0, 13.0 / 2.0),
                        doubleArrayOf(2.0, 1.0)
                    )
                )

                var FX: Matrix = x34.t().multiply(A).multiply(x34)
                FX = FX.scaled(-1.0 / 8.0)
                FX = FX.add(x34.t().multiply(B))

                var fx: Double = FX.get(1, 1)
                fx += -35.0 / 2.0

                return fx
            }

            override fun dimensionOfDomain(): Int {
                return 4
            }

            override fun dimensionOfRange(): Int {
                return 1
            }
        })

/**
* TODO: making the 2nd precision parameter 0 gives a better minimizer
* how to choose the precision parameters in general?
*/
// construct an SQP solver
val solver = SQPActiveSetOnlyInequalityConstraintMinimizer(
        1e-7, // epsilon1
        1e-3, // epsilon2
        10 // max number of iterations
)
// solving the SQP problem
val solution: IterativeSolution<Vector> = solver.solve(f, greater)
val x: Vector = solution.search(
        DenseVector(1.0, 0.5, 2.0, 3.0), // x0
        DenseVector(1.0, 1.0)) // μ0
val fx: Double = f.evaluate(x)
// print out the solution
println("x = " + x)
println("fx = " + fx)

Solving an SQP problem
x = [1.149349, -111.426292, 3.354936, -2.214956] 
fx = 5965.990172154221
